In [ ]:
# read electrode categories to see what proportion of electrodes are in SOZ 

In [3]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'FR1' # 'FR1' 'catFR1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# sub lists I've been using to explore FR1 

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2 # R1108J beatiful catFR1 data; R1065J magical subject (like with FR1)
# subs = ['R1112M','R1163T'] # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
# subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"
# subs = ['R1065J'] # subject with over 8000 FR1 trials X electrodes
# bad_subs = ['R1433E','R1051J'] # no electrode cats
# subs = ['R1379E','R1385E','R1387E','R1394E','R1402E'] # partial beep subs
# subs = ['R1379E','R1402E','R1396T','R1395M','R1415T','R1389J','R1404E']  # test subs for eeg offset correction
subs = ['R1048E','R1054J','R1345D','R1328E','R1308T',
        'R1137E','R1136N','R1094T','R1122E','R1385E'] # nice example FR1 subs. Both have electrode_categories in pwanda scratch too
# subs = ['R1048E']

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[0:6]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
239,NaN,protocols/r1/subjects/R1048E/experiments/FR1/s...,protocols/r1/subjects/R1048E/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1048E/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1048E/localizations/0/m...,NaN,0,R1048E,R1048E,NaN,protocols/r1/subjects/R1048E/experiments/FR1/s...
277,NaN,protocols/r1/subjects/R1054J/experiments/FR1/s...,protocols/r1/subjects/R1054J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1054J/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1054J/localizations/0/m...,NaN,0,R1054J,R1054J,NaN,protocols/r1/subjects/R1054J/experiments/FR1/s...
278,NaN,protocols/r1/subjects/R1054J/experiments/FR1/s...,protocols/r1/subjects/R1054J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1054J/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1054J/localizations/0/m...,NaN,1,R1054J,R1054J,NaN,protocols/r1/subjects/R1054J/experiments/FR1/s...
538,NaN,protocols/r1/subjects/R1094T/experiments/FR1/s...,protocols/r1/subjects/R1094T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1094T/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1094T/localizations/0/m...,NaN,0,R1094T,R1094T,1.0,protocols/r1/subjects/R1094T/experiments/FR1/s...
539,NaN,protocols/r1/subjects/R1094T/experiments/FR1/s...,protocols/r1/subjects/R1094T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1094T/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1094T/localizations/0/m...,NaN,1,R1094T,R1094T,NaN,protocols/r1/subjects/R1094T/experiments/FR1/s...
540,NaN,protocols/r1/subjects/R1094T/experiments/FR1/s...,protocols/r1/subjects/R1094T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1094T/experiments/FR1/s...,0,NaN,2,protocols/r1/subjects/R1094T/localizations/0/m...,NaN,2,R1094T,R1094T,NaN,protocols/r1/subjects/R1094T/experiments/FR1/s...
718,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...,protocols/r1/subjects/R1122E/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1122E/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1122E/localizations/0/m...,NaN,0,R1122E,R1122E,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...
719,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...,protocols/r1/subjects/R1122E/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1122E/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1122E/localizations/0/m...,NaN,1,R1122E,R1122E,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...
720,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...,protocols/r1/subjects/R1122E/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1122E/experiments/FR1/s...,0,NaN,2,protocols/r1/subjects/R1122E/localizations/0/m...,NaN,2,R1122E,R1122E,NaN,protocols/r1/subjects/R1122E/experiments/FR1/s...
786,NaN,protocols/r1/subjects/R1136N/experiments/FR1/s...,protocols/r1/subjects/R1136N/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1136N/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1136N/localizations/0/m...,NaN,0,R1136N,R1136N,NaN,protocols/r1/subjects/R1136N/experiments/FR1/s...


In [5]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
if exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df[404:413]
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(554, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
10,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,1,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...
2151,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,0,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,0,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2152,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,1,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,1,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2166,NaN,protocols/r1/subjects/R1466J/experiments/FR1/s...,protocols/r1/subjects/R1466J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1466J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1466J/localizations/0/m...,NaN,0,R1466J,R1466J,3.4,protocols/r1/subjects/R1466J/experiments/FR1/s...


In [41]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

### PARAMS ###

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal

### END PARAMS ###

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time


    
# things I want to know about SOZ:
has_L_SOZ = []
has_R_SOZ = []
regions_LR = []

for row in exp_df.itertuples(): #sub_df.itertuples():   
    sub = row.subject; session = row.session; exp = row.experiment
    mont = int(row.montage); loc = int(row.localization)
    reader = CMLReadDFRow(row)
    evs = reader.load('task_events')
    evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions).        
    word_evs = evs[evs['type']=='WORD'] # get words 

    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
    tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
    elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 

    # get bad channel mask
    try:
        elec_cats = reader.load('electrode_categories') # this is cool
    except:
        if remove_soz_ictal == True:
            e = 'No electrode categories for '+sub+', session '+str(session)
            LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')
            continue
        else:
            elec_cats = []

    # determine if SOZ sites in L and/or R hemisphere
    if elec_cats == []:
        has_R_SOZ.append(0)
        has_L_SOZ.append(0)
    else:
        soz_elecs = [elec[0] for elec in elec_cats['soz']]
        if 'R' in soz_elecs:
            has_R_SOZ.append(1)
        else:
            has_R_SOZ.append(0) 
        if 'L' in soz_elecs:
            has_L_SOZ.append(1)
        else:
            has_L_SOZ.append(0)

    bad_bp_mask = getBadChannels(tal_struct,elec_cats,remove_soz_ictal)

    electrode_search_range = getElectrodeRanges(elec_regions,exp,sub,session,mont)

    # how many L and R sites in our key regions?
    ca1_labels = [HPC_labels[label] for label in [0,5,14,17]]
    dg_labels = [HPC_labels[label] for label in [3,8,15,18]]

    temp = np.zeros(8)
    for channel in electrode_search_range:

        if (elec_regions[channel] in ca1_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[0]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[1]+=1

        if (elec_regions[channel] in dg_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L':
                temp[2]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[3]+=1

        if (elec_regions[channel] in ENT_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[4]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[5]+=1

        if (elec_regions[channel] in PHC_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[6]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[7]+=1

    regions_LR = superVstack(regions_LR,temp)

SOZ_LR = np.vstack([np.array(has_L_SOZ),np.array(has_R_SOZ)]).T

/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /data10/RAM/subjects/R1001P/docs/electrode_categories.txt
/scratch/pwanda/electrode_categories/electrode_categories_R1001P.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: 

In [60]:
# values
np.savetxt("../compiled/localization/FR1_CA1-DG-ENT-PHC_electrodes.csv",np.asarray(regions_LR), delimiter=",")
np.savetxt("../compiled/localization/FR1_sessions.csv",np.asarray(exp_df.session), delimiter=",")
np.savetxt("../compiled/localization/FR1_SOZ_LR.csv",np.asarray(SOZ_LR), delimiter=",")

# strings
np.savetxt("../compiled/localization/FR1_subs.csv",np.asarray(exp_df.subject), delimiter=",",fmt='%s')

## writing electrode_categories.txt (from Joey)

In [ ]:
import numpy as np
import pandas as pd
# read REDCap data
cat = pd.read_csv('REDCap_localization_12302020.csv')

# separate df by event type
resec = cat[cat['Event Name']=='Localization and Resection'].drop(columns=['Event Name', 'Study site'])
info = cat[cat['Event Name']=='Subject Consent & Surgery '][['Subject Number', 'Study site']]

# merge different REDCap "events" for the same subjects, so we have one row per subject
cat = pd.merge(resec, info, on='Subject Number', how='right')

# if empty, insert string "NONE"
cat = cat.replace(np.nan, 'NONE')

# dictionary lookup for generating subject codes of form R1###X
site_dict = {'University of Pennsylvania':'P', 'Dartmouth University':'D', 
             'Jefferson Hospital':'J', 'Emory University':'E', 'Mayo Clinic':'M', 
             'UT Southwestern':'T', 'Columbia University':'C', 'NINDS':'N', 'University of Washington':'W'}
cat['Subject Code'] = np.array(['R1']*len(cat)) + cat['Subject Number'].astype(str).str.zfill(3)+ cat['Study site'].apply(lambda x: site_dict[x])
cat = cat.drop(columns=['Subject Number', 'Study site'])

In [ ]:
for i, row in cat.iterrows():
    subject = row['Subject Code']
    # get lists of electrodes in each category
    ictal_onset = row[0].split('  ')
    interictal = row[1].split('  ')
    lesions = row[2].split('  ')
    broken = row[3].split('  ')
    # nice formatting with consistent section headers and spacing
    lines = [subject, '', 'Ictal Onset:'] + ictal_onset + ['', 'Interictal Spikes:'] + \
            interictal + ['', 'Brain Lesions:'] + lesions + ['', 'Broken Leads:'] + broken
    # add newlines
    lines = [x+'\n' for x in lines]
    # write to scratch (can choose another location of course)
    with open('/scratch/jrudoler/{}_electrode_categories.txt'.format(subject), 'w') as f:
        f.writelines(lines)